In [1]:
import os 

os.chdir("app/")

In [2]:
from functools import partial

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torch.nn as nn
import torch

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.trainer import Trainer

from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.data_load.datasets import MemoryMapDataset
from ptls.preprocessing import PandasDataPreprocessor

from ptls.nn import TrxEncoder
from ptls.nn.seq_encoder import RnnSeqEncoder

from ptls.frames import PtlsDataModule
from ptls.frames.coles.coles_module import CoLESModule
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices, NoSplit

from utils.encode import encode_data
from utils.evaluation import bootstrap_eval

## Load data

In [3]:
df = pd.read_parquet("data/preprocessed_new/default.parquet")
df.head()

,user_id,mcc_code,amount,timestamp,holiday_target,weekend_target,global_target,default_target
0,69,5,-342.89792,2021-03-05 02:52:36,0,0,0,0
1,69,21,-1251.88120,2021-03-05 09:43:28,0,0,0,0
2,69,12,-87.30924,2021-03-05 11:17:23,0,0,0,0
3,69,6,-1822.17700,2021-03-05 13:41:03,0,0,0,0
4,69,18,-427.12363,2021-03-05 19:14:23,0,0,0,0


## Prepare dataset and dataloaders

In [4]:
preprocessor = PandasDataPreprocessor(
    col_id="user_id",
    col_event_time="timestamp",
    event_time_transformation="dt_to_timestamp",
    cols_category=["mcc_code"],
    cols_first_item=["global_target"]
)

In [5]:
data = preprocessor.fit_transform(df)

In [15]:
y = [data[i]["global_target"] for i in range(len(data))] 

In [23]:
val_size = 0.1
test_size = 0.1

train, val_test, y_train, y_val_test = train_test_split(data, y, stratify=y, test_size=test_size+val_size, random_state=42)
val, test, y_val, y_test = train_test_split(val_test, y_val_test, stratify=y_val_test, test_size=test_size/(test_size+val_size), random_state=42)

In [24]:
train_ds = ColesDataset(
    data=MemoryMapDataset(train, [SeqLenFilter(min_seq_len=15)]),
    splitter=SampleSlices(5, 15, 150),
)
val_ds = ColesDataset(
    data=MemoryMapDataset(val, [SeqLenFilter(min_seq_len=15)]),
    splitter=SampleSlices(5, 15, 150),
)
test_ds = MemoryMapDataset(test, [SeqLenFilter(min_seq_len=15)])

In [25]:
datamodule = PtlsDataModule(
    train_data=train_ds,
    valid_data=val_ds,
    train_batch_size=128,
    valid_batch_size=128,
    train_num_workers=8,
    valid_num_workers=8
)

# CoLES

## Model training

In [26]:
trx_encoder = TrxEncoder(
    embeddings={
        "mcc_code": {"in": 345, "out": 24}
    },
    numeric_values={
        "amount": "identity",
    },
    use_batch_norm_with_lens=True,
    norm_embeddings=False,
    embeddings_noise=0.0003
)

seq_encoder = RnnSeqEncoder(
    trx_encoder,
    hidden_size=1024,
    type="lstm",
    bidir=False,
    trainable_starter="static",
)

In [27]:
lr_scheduler_partial = partial(torch.optim.lr_scheduler.ReduceLROnPlateau, factor=.9025, patience=5, mode="max")
optimizer_partial = partial(torch.optim.Adam, lr=4e-3)

model = CoLESModule(
    seq_encoder,
    optimizer_partial=optimizer_partial,
    lr_scheduler_partial=lr_scheduler_partial
)

In [28]:
checkpoint = ModelCheckpoint(
    monitor="recall_top_k", 
    mode="max"
)

trainer = Trainer(
    max_epochs=50,
    devices=[1],
    accelerator="gpu",
    callbacks=[checkpoint]
)

trainer.fit(model, datamodule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name               | Type            | Params
-------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0     
1 | _seq_encoder       | RnnSeqEncoder   | 4.3 M 
2 | _validation_metric | BatchRecallTopK | 0     
3 | _head              | Head            | 0     
-------------------------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.257    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/macro-micro-coles/miniconda/envs/env/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1933: PossibleUserWarning: The number of training batches (45) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [34]:
model.load_state_dict(torch.load(checkpoint.best_model_path)["state_dict"])

<All keys matched successfully>

In [41]:
# torch.save(model.seq_encoder.state_dict(), "coles_churn.pth")

## Model evaluation

In [29]:
train_val_ds = MemoryMapDataset(train + val, [SeqLenFilter(min_seq_len=15)])

In [30]:
X_train, y_train = encode_data(model.seq_encoder, train_val_ds)
X_test, y_test = encode_data(model.seq_encoder, test_ds)

print("Train size:", len(y_train))
print("Test size:", len(y_test))

Train size: 6372
Test size: 708


In [31]:
from sklearn.metrics import roc_auc_score, average_precision_score
from lightgbm import LGBMClassifier


lgbm = LGBMClassifier(
    n_estimators=500,
    boosting_type="gbdt",
    subsample=0.5,
    subsample_freq=1,
    learning_rate=0.02,
    feature_fraction=0.75,
    max_depth=6,
    lambda_l1=1,
    lambda_l2=1,
    min_data_in_leaf=50,
    random_state=42,
    n_jobs=8,
    verbose=-1
)

lgbm.fit(X_train, y_train)

LGBMClassifier(feature_fraction=0.75, lambda_l1=1, lambda_l2=1,
               learning_rate=0.02, max_depth=6, min_data_in_leaf=50,
               n_estimators=500, n_jobs=8, random_state=42, subsample=0.5,
               subsample_freq=1, verbose=-1)

In [32]:
y_pred = lgbm.predict_proba(X_test)[:, 1]

roc_auc_score(y_test, y_pred), average_precision_score(y_test, y_pred)

(0.6130724114595083, 0.0640716599343085)

# CoLES with time features

## Model training

In [9]:
trx_encoder = TrxEncoder(
    embeddings={
        "mcc_code": {"in": 345, "out": 24}
    },
    numeric_values={
        "amount": "identity",
        "event_time": "identity",
        "time_delta": "identity",
    },
    use_batch_norm_with_lens=True,
    norm_embeddings=False,
    embeddings_noise=0.0003
)

seq_encoder = RnnSeqEncoder(
    trx_encoder,
    hidden_size=1024,
    type="lstm",
    bidir=False,
    trainable_starter="static",
)

In [10]:
lr_scheduler_partial = partial(torch.optim.lr_scheduler.ReduceLROnPlateau, factor=.9025, patience=5, mode="max")
optimizer_partial = partial(torch.optim.Adam, lr=4e-3)

model = CoLESModule(
    seq_encoder,
    optimizer_partial=optimizer_partial,
    lr_scheduler_partial=lr_scheduler_partial
)

In [ ]:
checkpoint = ModelCheckpoint(
    monitor="recall_top_k", 
    mode="max"
)

trainer = Trainer(
    max_epochs=50,
    devices=[1],
    accelerator="gpu",
    callbacks=[checkpoint]
)

trainer.fit(model, datamodule)

In [21]:
model.load_state_dict(torch.load(checkpoint.best_model_path)["state_dict"])

<All keys matched successfully>

In [30]:
torch.save(model.seq_encoder.state_dict(), "coles_churn_date.pth")

## Model evaluation

In [24]:
train_val_ds = MemoryMapDataset(train + val, [SeqLenFilter(min_seq_len=15)])

In [25]:
X_train, y_train = encode_data(model.seq_encoder, train_val_ds)
X_test, y_test = encode_data(model.seq_encoder, test_ds)

print("Train size:", len(y_train))
print("Test size:", len(y_test))

Train size: 3961
Test size: 443


In [26]:
results = bootstrap_eval(X_train, X_test, y_train, y_test, n_runs=10)

In [27]:
results

,ROC-AUC,PR-AUC
0,0.797338,0.863472
1,0.805485,0.871617
2,0.812807,0.874014
3,0.803771,0.867662
4,0.806120,0.854886
5,0.800406,0.859777
6,0.798713,0.858944
7,0.812807,0.873045
8,0.815177,0.872571
9,0.806882,0.862757


In [29]:
results.agg(["mean", "std"])

,ROC-AUC,PR-AUC
mean,0.805951,0.865875
std,0.006167,0.006833
